# Melee Replay Exploration

Initial exploration of .slp replay data using melee-tools.

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from melee_tools.parse import parse_directory
from melee_tools.stats import game_stats_directory
from melee_tools.frames import extract_frames
from melee_tools.action_states import ACTION_STATES, ACTION_STATE_CATEGORIES, FRIENDLY_NAMES

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
sns.set_theme(style='whitegrid')

REPLAY_DIR = '../replays'

## 1. Game-Level Overview

In [ ]:
df = game_stats_directory(REPLAY_DIR)

# Filter to real games (not NO_CONTEST quitouts)
games = df[df['end_method'].isin(['RESOLVED', 'GAME'])].copy()
print(f'Total replays: {len(df)}, Real games: {len(games)}')
print(f'Teams: {games["is_teams"].sum()}, 1v1: {(~games["is_teams"]).sum()}')
print(f'\nStage distribution:')
print(games['stage_name'].value_counts())

In [ ]:
# Character frequency across all player slots
chars = []
for _, row in games.iterrows():
    for i in range(int(row['num_players'])):
        chars.append(row.get(f'p{i}_character'))

char_counts = pd.Series(chars).value_counts()

fig, ax = plt.subplots(figsize=(10, 5))
char_counts.plot(kind='barh', ax=ax)
ax.set_xlabel('Times Played')
ax.set_title('Character Frequency (all games)')
plt.tight_layout()
plt.show()

## 2. Player Performance (by name tag)

In [ ]:
# Build a player-level view from game stats
player_rows = []
for _, row in games.iterrows():
    for i in range(int(row['num_players'])):
        p = f'p{i}'
        player_rows.append({
            'filename': row['filename'],
            'name_tag': row.get(f'{p}_name_tag') or 'Unknown',
            'character': row[f'{p}_character'],
            'placement': row.get(f'{p}_placement'),
            'damage_dealt': row.get(f'{p}_damage_dealt', 0),
            'damage_received': row.get(f'{p}_damage_received', 0),
            'stocks_lost': row.get(f'{p}_stocks_lost', 0),
            'l_cancel_rate': row.get(f'{p}_l_cancel_rate'),
            'is_teams': row['is_teams'],
            'stage': row['stage_name'],
            'duration_sec': row['duration_seconds'],
        })

players = pd.DataFrame(player_rows)
players['won'] = players['placement'] == 0
players.head(10)

In [ ]:
# Stats by name tag
by_tag = players.groupby('name_tag').agg(
    games_played=('filename', 'count'),
    wins=('won', 'sum'),
    avg_damage_dealt=('damage_dealt', 'mean'),
    avg_damage_received=('damage_received', 'mean'),
    avg_stocks_lost=('stocks_lost', 'mean'),
    avg_l_cancel_rate=('l_cancel_rate', 'mean'),
).round(2)
by_tag['win_rate'] = (by_tag['wins'] / by_tag['games_played']).round(3)
by_tag.sort_values('games_played', ascending=False)

## 3. Damage Dealt vs Received

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
for tag in players['name_tag'].unique():
    subset = players[players['name_tag'] == tag]
    ax.scatter(subset['damage_dealt'], subset['damage_received'],
              label=tag, alpha=0.7, s=60)

# Add diagonal line (break-even)
lims = [0, max(players['damage_dealt'].max(), players['damage_received'].max()) + 50]
ax.plot(lims, lims, 'k--', alpha=0.3, label='Break-even')

ax.set_xlabel('Damage Dealt')
ax.set_ylabel('Damage Received')
ax.set_title('Damage Dealt vs Received (per game)')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 4. L-Cancel Rates by Character

In [ ]:
lc = players[players['l_cancel_rate'].notna()].copy()

fig, ax = plt.subplots(figsize=(10, 5))
order = lc.groupby('character')['l_cancel_rate'].mean().sort_values(ascending=False).index
sns.boxplot(data=lc, x='character', y='l_cancel_rate', order=order, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_ylabel('L-Cancel Rate')
ax.set_title('L-Cancel Rate by Character')
plt.tight_layout()
plt.show()

## 5. Frame-Level Analysis: Post-Hitstun Options

Example of a conditional state query: after getting hit, what do players do?

In [ ]:
def post_hitstun_actions(filepath, player_index):
    """What action does a player take after leaving hitstun?"""
    result = extract_frames(filepath, include_inputs=False)
    df = result['players'][player_index]
    
    damage_states = ACTION_STATE_CATEGORIES['damage']
    in_hitstun = df['state'].isin(damage_states)
    hitstun_ends = in_hitstun & ~in_hitstun.shift(-1, fill_value=False)
    
    actions = []
    for idx in df.index[hitstun_ends]:
        next_idx = idx + 1
        if next_idx in df.index:
            state = int(df.loc[next_idx, 'state'])
            name = FRIENDLY_NAMES.get(state, ACTION_STATES.get(state, f'Unknown({state})'))
            actions.append(name)
    
    return pd.Series(actions).value_counts()

# Analyze post-hitstun for a 1v1 game
game_file = f'{REPLAY_DIR}/Game_20250427T232807.slp'
result = extract_frames(game_file, include_inputs=False)

print('G&W post-hitstun actions:')
print(post_hitstun_actions(game_file, 0))
print()
print('Jigglypuff post-hitstun actions:')
print(post_hitstun_actions(game_file, 1))

## 6. Position Heatmap

Where do players spend their time on stage?

In [ ]:
result = extract_frames(f'{REPLAY_DIR}/Game_20250427T232807.slp', include_inputs=False)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for idx, ax in enumerate(axes):
    df = result['players'][idx]
    char = df['character_name'].iloc[0]
    
    # Filter to alive frames
    alive = df[df['stocks'] > 0]
    
    ax.hexbin(alive['position_x'], alive['position_y'],
             gridsize=30, cmap='YlOrRd', mincnt=1)
    ax.set_xlabel('X Position')
    ax.set_ylabel('Y Position')
    ax.set_title(f'{char} Position Heatmap')
    ax.set_aspect('equal')

plt.suptitle('Fountain of Dreams — Player Positions', y=1.02)
plt.tight_layout()
plt.show()

## 7. Damage Over Time (Single Game)

In [ ]:
result = extract_frames(f'{REPLAY_DIR}/Game_20250427T232807.slp', include_inputs=False)

fig, ax = plt.subplots(figsize=(12, 4))
for idx in result['players']:
    df = result['players'][idx]
    char = df['character_name'].iloc[0]
    seconds = df['frame'] / 60
    ax.plot(seconds, df['percent'], label=char, alpha=0.8)

ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Damage %')
ax.set_title('Damage % Over Time')
ax.legend()
plt.tight_layout()
plt.show()